In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

#Support vector machines 
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "RNF": RandomForestClassifier(random_state=1234),
    "XGB": XGBClassifier(use_label_encoder=False),
    "CAT": CatBoostClassifier(random_state=1234),
    "ADA": AdaBoostClassifier(SVC(random_state=1234, kernel='rbf', probability=True)),
    "BAG": BaggingClassifier(SVC(random_state=1234, kernel='rbf', probability=True))
}

import dalex as dx
import pickle as pk

dataset = 'adult'

import worstcase_helper
import importlib
importlib.reload(worstcase_helper)

preprocess, X, y = worstcase_helper.load_dataset_with_preprocess(dataset)

from sklearn.pipeline import make_pipeline
def make_pipeline_clf(clf_name):
    clf = make_pipeline(
        preprocess,
        clfs[clf_name]
    )
    return clf

clf_cart = pk.load(open(f"./models/{dataset}_cart", 'rb'))
# clf_cart.fit(X, y)

clf_rnf = pk.load(open(f"./models/{dataset}_rnf", 'rb'))
# clf_rnf = make_pipeline_clf("RNF")
# clf_rnf.fit(X, y)
clf_xgb = pk.load(open(f"./models/{dataset}_xgb", 'rb'))
# clf_xgb = make_pipeline_clf("XGB")
# clf_xgb.fit(X, y)
clf_cat = pk.load(open(f"./models/{dataset}_cat", 'rb'))
# clf_cat = make_pipeline_clf("CAT")
# clf_cat.fit(X, y)
clf_ada = pk.load(open(f"./models/{dataset}_ada", 'rb'))
# clf_ada = make_pipeline_clf("ADA")
# clf_ada.fit(X, y)
clf_bag = pk.load(open(f"./models/{dataset}_bag", 'rb'))
# clf_bag = make_pipeline_clf("BAG")
# clf_bag.fit(X, y)

clf_cart_exp = dx.Explainer(clf_cart, X, y, label="CART")
clf_ebm_exp = dx.Explainer(clf_rnf, X, y, label="RNF")
clf_lr_l2_exp = dx.Explainer(clf_xgb, X, y, label="XGB")
clf_gnb_exp = dx.Explainer(clf_cat, X, y, label="CAT")
clf_lr_exp = dx.Explainer(clf_ada, X, y, label="ADA")
clf_dl_exp = dx.Explainer(clf_bag, X, y, label="BAG")



ModuleNotFoundError: No module named 'cuml'

In [ ]:
cat_feat = ['WorkClass', 'Education', 'MaritalStatus',
                                  'Occupation', 'Relationship', 'Race',
                                  'Gender', 'NativeCountry']
cont_feat = ['Age', 'fnlwgt', 'EducationNum',
                                  'CapitalGain', 'CapitalLoss',
                                  'HoursPerWeek']

In [ ]:
#categorical pd profiles
pd_cart_cat = clf_cart_exp.model_profile(variable_type="categorical", variables=cat_feat)

pd_ebm_cat = clf_ebm_exp.model_profile(variables= cat_feat,
                                         variable_type="categorical")

pd_lr_l2_cat = clf_lr_l2_exp.model_profile(variables= cat_feat,
                                         variable_type="categorical")

pd_gnb_cat = clf_gnb_exp.model_profile(variables= cat_feat,
                                         variable_type="categorical")

pd_lr_cat = clf_lr_exp.model_profile(variables= cat_feat,
                                         variable_type="categorical")

pd_dl_cat = clf_dl_exp.model_profile(variables= cat_feat,
                                         variable_type="categorical")

#continous pd profiles
pd_cart = clf_cart_exp.model_profile(variables= cont_feat)

pd_ebm = clf_ebm_exp.model_profile(variables= cont_feat)

pd_lr_l2 = clf_lr_l2_exp.model_profile(variables= cont_feat)

pd_gnb = clf_gnb_exp.model_profile(variables= cont_feat)

pd_lr = clf_lr_exp.model_profile(variables= cont_feat)

pd_dl = clf_dl_exp.model_profile(variables= cont_feat)